## Import necessary packages

In [1]:
import scipy.io
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef, hamming_loss

from keras.models import Sequential
from keras import regularizers
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping#, ReduceLROnPlateau

np.random.seed(123)

Using TensorFlow backend.


## Reading files

In [2]:
data = scipy.io.loadmat('../datasets/P_M_S3_Remove15.mat')  #Rand_3pairs_multi.mat 

X = data['X_all'] # Input (Reflectance Spectrum)
Y_multi = data['Y_multi_all'] # Output Vectors

## Splitting data to training, validation and test sets

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, Y_multi, test_size=0.2, random_state=1) #0.6
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) #0.6

## Maybe I need to normalize the data
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')
#X_train /= 4095
#X_val /= 4095
#X_test /= 4095

## Defining the model

In [4]:
# create model
model = Sequential()
model.add(Dense(256, input_dim=226, activation='sigmoid'))
#model.add(Dense(128, input_dim=221))
#model.add(Dense(64, input_dim=221))#, activation='sigmoid'))
#model.add(Dropout(0.25))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
#model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32))#, activation='sigmoid'))
#model.add(Dropout(0.25))
model.add(Dense(16))
model.add(Dense(Y_multi.shape[1], activation='sigmoid'))

In [5]:
early_checker = EarlyStopping(monitor='val_acc', patience=5, verbose=0, mode='auto')

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
nb_epoch = 100
# Fit the model
model.fit(X_train, y_train, epochs=nb_epoch, batch_size=32, verbose=1, validation_data=(X_val, y_val), callbacks=[early_checker]) #early_checker

Train on 10560 samples, validate on 2640 samples
Epoch 1/100
10560/10560 [==============================] - 3s 278us/step - loss: 0.2912 - acc: 0.8728 - val_loss: 0.1368 - val_acc: 0.9479
Epoch 2/100
10560/10560 [==============================] - 2s 217us/step - loss: 0.0760 - acc: 0.9763 - val_loss: 0.0344 - val_acc: 0.9941
Epoch 3/100
10560/10560 [==============================] - 2s 225us/step - loss: 0.0205 - acc: 0.9968 - val_loss: 0.0105 - val_acc: 0.9987
Epoch 4/100
10560/10560 [==============================] - 2s 226us/step - loss: 0.0088 - acc: 0.9983 - val_loss: 0.0065 - val_acc: 0.9986
Epoch 5/100
10560/10560 [==============================] - 3s 240us/step - loss: 0.0064 - acc: 0.9984 - val_loss: 0.0051 - val_acc: 0.9993
Epoch 6/100
10560/10560 [==============================] - 3s 248us/step - loss: 0.0049 - acc: 0.9987 - val_loss: 0.0036 - val_acc: 0.9997
Epoch 7/100
10560/10560 [==============================] - 3s 247us/step - loss: 0.0042 - acc: 0.9989 - val_loss: 0.0

## evaluate the model

In [7]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.00328477535408
Test accuracy: 0.99929293199


In [8]:
"""
# calculate predictions
predictions = model.predict(X_test)
"""
predictions = model.predict_proba(X_test)
threshold = np.arange(0.1,0.9,0.05)

acc = []
accuracies = []
best_threshold = np.zeros(predictions.shape[1])
for i in range(predictions.shape[1]):
    acc = []
    y_prob = np.array(predictions[:,i])
    for j in threshold:
        y_pred = [1 if prob>=j else 0 for prob in y_prob]
        acc.append( matthews_corrcoef(y_test[:,i],y_pred))
    acc   = np.array(acc)
    index = np.where(acc==acc.max())
    accuracies.append(acc.max())
    best_threshold[i] = threshold[index[0][0]]


print (best_threshold)

3300/3300 [==============================] - 0s 47us/step
[ 0.1   0.1   0.1   0.1   0.45  0.1   0.1   0.1   0.85]


In [9]:
y_pred = np.array([[1 if predictions[i,j]>=best_threshold[j] else 0 for j in range(y_test.shape[1])] for i in range(len(y_test))])
print (hamming_loss(y_test,y_pred))

total_correctly_predicted = len([i for i in range(len(y_test)) if (y_test[i]==y_pred[i]).sum() == Y_multi.shape[1]])
print (float(total_correctly_predicted)/X_test.shape[0], total_correctly_predicted, X_test.shape[0])

0.00037037037037037035
0.9966666666666667 3289 3300


In [10]:
"""
# calculate predictions
predictions = model.predict(X_test)
"""
## Using validation to find the best thresholds

predictions_val = model.predict_proba(X_val)
threshold_val = np.arange(0.1,0.9,0.05)

acc_val = []
accuracies_val = []
best_threshold_val = np.zeros(predictions_val.shape[1])
for i in range(predictions_val.shape[1]):
    acc_val = []
    y_prob_val = np.array(predictions_val[:,i])
    for j in threshold_val:
        y_pred_val = [1 if prob>=j else 0 for prob in y_prob_val]
        acc_val.append( matthews_corrcoef(y_val[:,i],y_pred_val))
    acc_val   = np.array(acc_val)
    index = np.where(acc_val==acc_val.max())
    accuracies_val.append(acc_val.max())
    best_threshold_val[i] = threshold_val[index[0][0]]


print (best_threshold_val)

2640/2640 [==============================] - 0s 40us/step
[ 0.1   0.1   0.1   0.1   0.5   0.1   0.1   0.1   0.65]


In [11]:
y_pred2 = np.array([[1 if predictions[i,j]>=best_threshold_val[j] else 0 for j in range(y_test.shape[1])] for i in range(len(y_test))])

print (hamming_loss(y_test,y_pred2))

total_correctly_predicted2 = len([i for i in range(len(y_test)) if (y_test[i]==y_pred2[i]).sum() == Y_multi.shape[1]])

print (float(total_correctly_predicted2)/X_test.shape[0], total_correctly_predicted2, X_test.shape[0])

0.00047138047138047136
0.9957575757575757 3286 3300


In [12]:
## Reading files
data = scipy.io.loadmat('../datasets/Manu_L_Box_Sm.mat')  #Islamic_img2.mat

X_real = data['smooth_Im'] #img_mat

In [13]:
predictions_real = model.predict_proba(np.transpose(X_real))

114144/114144 [==============================] - 4s 36us/step


In [14]:
num_classes = Y_multi.shape[1]

In [15]:
y_pred_real = np.array([[1 if predictions_real[i,j]>=best_threshold[j] else 0 for j in range(num_classes)] for i in range(X_real.shape[1])])
y_pred2_real = np.array([[1 if predictions_real[i,j]>=best_threshold_val[j] else 0 for j in range(num_classes)] for i in range(X_real.shape[1])])

In [16]:
scipy.io.savemat('../datasets/Manuscript/P_M/y_pred_manu_L_box_sm.mat', mdict={'y_pred_real': y_pred_real})
scipy.io.savemat('../datasets/Manuscript/P_M/y_pred2_manu_L_box_sm.mat', mdict={'y_pred2_real': y_pred2_real} )